# Analysis

This notebook provides several analyses to gain deeper intuition into system performance.

In [ ]:
%matplotlib inline

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import librosa as lb
import pandas as pd
import pickle
import soundfile as sf
import IPython.display as ipd
import os.path

## Visualizing errors over time

The first analysis is to simply visualize the magnitude of alignment errors over time.

In [ ]:
EVAL_DIR = 'eval/simpleOfflineDTW' # eval directory to visualize

In [ ]:
with open(f'{EVAL_DIR}/errs.pkl', 'rb') as f:
    d = pickle.load(f)

In [ ]:
for scenario_id in d:
    errs = d[scenario_id][0] # downbeat alignment errors 
    measures = len(errs)
    plt.plot(np.arange(measures), errs)
plt.xlabel('Measure Number')
plt.ylabel('Alignment Error (sec)')

There are large errors at the beginning.  This should not be happening, since we assume that the pianist comes in at the right time.

## Generating sonifications

The second analysis is to use the estimated alignment to time-scaled modify the orchestral recording.  We generate an audio file that contains the original piano recording on the left channel and the time-scale modified accompaniment on the right channel.

In [ ]:
import import_ipynb
import tsm_tools

In [ ]:
def mix_separate_channels(left_channel, right_channel):
    '''
    Merges two mono audio waveforms into a stereo audio waveform.  If the two waveforms differ
    in length, the longer of the two is truncated.
    
    Inputs
    left_channel: the audio waveform for the left channel
    right_channel: the audio waveform for the right channel
    '''
    N = min(len(left_channel), len(right_channel))
    mixed = np.zeros((N, 2))
    mixed[:,0] = left_channel[0:N]
    mixed[:,1] = right_channel[0:N]
    return mixed

In [ ]:
def sonifyWithAccompaniment(piano_audiofile, orch_audiofile, align_file, downsample, hop_len, outfile = None):
    '''
    Generates a stereo audio recording with the original piano recording on one channel
    and the time-scale modified orchestral recording on the other channel.
    
    Inputs
    piano_audiofile: filepath to the audio recording containing the piano (only) part
    orch_audiofile: filepath to the audio recording containing the orchestra (only) part
    align_file: filepath to the .npy file specifying the warping path between piano and orchestra parts
    downsample: downsample the warping path by this factor to smooth out the TSM
    hop_len: specifies the hop length in seconds between frames, needed to convert warping path to timestamps
    outfile: the output audio file to generate
    '''
    y_piano, sr = lb.load(piano_audiofile)
    y_orch, sr = lb.load(orch_audiofile)
    wp = np.load(align_file) # 2xN array specifying piano-orchestra alignment (in frames)
    wp = np.flipud(wp[:,0::downsample]) * hop_len # convert to sec, downsample for smoothing
    y_orch_tsm = tsm_tools.tsmvar_hybrid(y_orch, wp)
    y_mixed = mix_separate_channels(y_piano, y_orch_tsm)
    if outfile:
        sf.write(outfile, y_mixed, sr, subtype='PCM_24')
    return y_mixed

Here is example usage for sonifying a single scenario:

In [ ]:
# piano_file = f'scenarios/s1/p.wav'
# orch_file = 'scenarios/s1/o.mp3'
# align_file = 'experiments/simpleOfflineDTW/s1/hyp.npy'
# downsample = 20
# sr = 22050
# hop_len = 512./sr
# y = sonifyWithAccompaniment(piano_file, orch_file, align_file, downsample, hop_len)
# ipd.Audio(y, rate=sr)

In [ ]:
def sonifyWithAccompaniment_batch(scenarios_dir, exp_dir, downsample, hop_len, outdir):
    '''
    Generates stereo recordings of piano (left channel) and time-scale modified orchestra recordings
    (right channel) for all scenarios.
    
    Inputs
    scenarios_dir: directory containing all scenario directories
    exp_dir: directory containing all the hypothesis alignments
    downsample: downsample the warping path by this factor to smooth out the TSM
    hop_len: specifies the hop length in seconds between frames, needed to convert warping path to timestamps
    outdir: directory to put the generated audio files
    '''
    assert not os.path.exists(outdir)
    os.makedirs(outdir)
    
    scenario_ids = list(pd.read_csv(f'{scenarios_dir}/scenarios.summary', header=None, sep=' ')[0])
    for scenario_id in scenario_ids:
        piano_file = f'{scenarios_dir}/{scenario_id}/p.wav'
        orch_file = f'{scenarios_dir}/{scenario_id}/o.mp3'
        align_file = f'{exp_dir}/{scenario_id}/hyp.npy'
        out_file = f'{outdir}/{scenario_id}.wav'
        sonifyWithAccompaniment(piano_file, orch_file, align_file, downsample, hop_len, out_file)
        
    return

The following two cells generate sonifications for all scenarios:

In [ ]:
SCENARIOS_DIR = 'scenarios'
EXP_DIR = 'experiments/simpleOfflineDTW'
SONIFY_DIR = f'{EXP_DIR}/sonify'
downsample = 20
hop_len = 512./22050

In [ ]:
sonifyWithAccompaniment_batch(SCENARIOS_DIR, EXP_DIR, downsample, hop_len, SONIFY_DIR)

Listen to a selected example:

In [ ]:
ipd.Audio(f'{SONIFY_DIR}/s1.wav')